In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import roi
import cv2
from skimage import io, draw, morphology
import os
import glob
import numpy as np

In [16]:
#roi_dir = '/run/media/dhruv/Hufflepuff/Data/20-9-18/HEK293EH187prefors_1/ROI/'
roi_dir = '/home/dhruv/Cybergut/Data/09-10-18/rois/HEK293E_hGCC_FG3_NoST/'
mask_dir = '/home/dhruv/Cybergut/Data/09-10-18/HEK293E_hGCC_FG3_48WP/HEK293E_hGCC_FG3_NoST/HEK293E_hGCC_FG3_NoST_9/Masks'

rois = glob.glob(os.path.join(roi_dir, '*.zip'))

# Black & White

# Color

In [2]:
BG = 0
FG = 1
BD = 2

In [8]:
rois = glob.glob(os.path.join(roi_dir, '*.zip'))
rois[:2]

['/home/dhruv/Cybergut/Data/09-10-18/rois/HEK293E_hGCC_FG3_NoST/A - 1(fld 06 wv TL-Brightfield - dsRed).zip',
 '/home/dhruv/Cybergut/Data/09-10-18/rois/HEK293E_hGCC_FG3_NoST/A - 1(fld 01 wv TL-Brightfield - dsRed).zip']

In [19]:
count = [0,0,0]
for r in rois:
    base = os.path.basename(r)
    base = base.replace('.zip', '.png')
    print(r)
    img = roi.create_color_mask(roi.get_polygons(r), 2048, 5, False)
    cv2.imwrite(os.path.join(mask_dir, base), img)
    nimg = img.argmax(axis=2)
    u, c = np.unique(nimg, return_counts=True)
    for i in range(3):
        count[u[i]] += c[i]

/home/dhruv/Cybergut/Data/09-10-18/rois/HEK293E_hGCC_FG3_NoST/A - 1(fld 06 wv TL-Brightfield - dsRed).zip
/home/dhruv/Cybergut/Data/09-10-18/rois/HEK293E_hGCC_FG3_NoST/A - 1(fld 01 wv TL-Brightfield - dsRed).zip
/home/dhruv/Cybergut/Data/09-10-18/rois/HEK293E_hGCC_FG3_NoST/A - 1(fld 08 wv TL-Brightfield - dsRed).zip
/home/dhruv/Cybergut/Data/09-10-18/rois/HEK293E_hGCC_FG3_NoST/A - 1(fld 03 wv TL-Brightfield - dsRed).zip
/home/dhruv/Cybergut/Data/09-10-18/rois/HEK293E_hGCC_FG3_NoST/A - 1(fld 04 wv TL-Brightfield - dsRed).zip
/home/dhruv/Cybergut/Data/09-10-18/rois/HEK293E_hGCC_FG3_NoST/A - 1(fld 07 wv TL-Brightfield - dsRed).zip


In [13]:
count

[37059486, 17648673, 4012097]

In [18]:
count

[37059486, 17648673, 4012097]

In [22]:
bg, fg, bd = count
bg > fg, bg > bd, fg > bd

(True, True, True)

In [24]:
n = bg+fg+bd
bg/n, fg/n, bd/n

(0.6311192853110177, 0.30055510997772217, 0.06832560471126012)

In [27]:
.07+.30+.63

1.0

# Separate borders for each Cell, 512x512 patches
To be used for creating weightmaps

In [10]:
#mask_dir = '/run/media/dhruv/Hufflepuff/Data/20-9-18/processed/groundtruth/512Patches/boundries1'
mask_dir='/home/dhruv/Cybergut/Data/09-10-18/patches/Boundries'
os.makedirs(mask_dir,exist_ok=True)

In [11]:
rois[:2]

['/run/media/dhruv/Hufflepuff/Data/20-9-18/HEK293EH187prefors_1/ROI/B - 3(fld 01 wv TL-Brightfield - dsRed).zip',
 '/run/media/dhruv/Hufflepuff/Data/20-9-18/HEK293EH187prefors_1/ROI/B - 3(fld 02 wv TL-Brightfield - dsRed).zip']

In [12]:
for r in rois:
    boundaries = []
    base = os.path.basename(r)[:-4]
    print(f"Processing {base}")
    polies = roi.get_polygons(r, True)
    xys = [(d['x'], d['y']) for d in polies]
    mask, bdrs = roi.create_maskv2(xys, 2048, True, 2)
    dist = 256
    npatches_h = 7
    npatches_v = 7
    basedir = os.path.join(mask_dir, base)
    os.makedirs(basedir,exist_ok=True)
    i = -1
    for r in (dist*i for i in range(npatches_v)):
        for c in (dist*i for i in range(npatches_h)):
            i += 1
            subbasepath = os.path.join(basedir, f'{base}_{i}')
            os.makedirs(subbasepath, exist_ok=True)
            j = 0
            crop = mask[r:r+512, c:c+512]
            cv2.imwrite(subbasepath+'/mask.png',crop)
            for bdr, poly in zip(bdrs, polies):
                top, left = poly['tl']
                bottom, right = poly['br']
                if right < c or left >= c+512 or bottom < r or top >= r+512:
                    continue
                crop = bdr[r:r+512, c:c+512]
                cv2.imwrite(subbasepath + f'/{j}.png', crop)
                j += 1

Processing B - 3(fld 01 wv TL-Brightfield - dsRed)
Processing B - 3(fld 02 wv TL-Brightfield - dsRed)
Processing B - 3(fld 03 wv TL-Brightfield - dsRed)
Processing B - 3(fld 04 wv TL-Brightfield - dsRed)
Processing B - 3(fld 05 wv TL-Brightfield - dsRed)
Processing B - 3(fld 06 wv TL-Brightfield - dsRed)
Processing B - 3(fld 07 wv TL-Brightfield - dsRed)
Processing B - 3(fld 08 wv TL-Brightfield - dsRed)
Processing B - 3(fld 09 wv TL-Brightfield - dsRed)
Processing B - 3(fld 10 wv TL-Brightfield - dsRed)
Processing B - 3(fld 11 wv TL-Brightfield - dsRed)
Processing B - 3(fld 12 wv TL-Brightfield - dsRed)
Processing B - 3(fld 13 wv TL-Brightfield - dsRed)
Processing B - 3(fld 14 wv TL-Brightfield - dsRed)


# Color with DeadCells as well as Halfcells and original

In [4]:
roi_dir = '/run/media/dhruv/Hufflepuff/Data/20-9-18/HEK293EH187prefors_1/ROI/all'
mask_dir = ''

In [6]:
rois = glob.glob(os.path.join(roi_dir, '*.zip'))
rois[:2]

['/run/media/dhruv/Hufflepuff/Data/20-9-18/HEK293EH187prefors_1/ROI/all/B - 3(fld 01 wv TL-Brightfield - dsRed).zip',
 '/run/media/dhruv/Hufflepuff/Data/20-9-18/HEK293EH187prefors_1/ROI/all/B - 3(fld 02 wv TL-Brightfield - dsRed).zip']

In [10]:
for r in rois:
    base = os.path.basename(r)
    base = base.replace('.zip', '.png')
    img = roi.create_color_mask(roi.get_polygons(r), 2048, 5, False)
    cv2.imwrite(os.path.join('/tmp', base), img)

(0.57025865146092, 0.3462242569242205, 0.08351709161485944)

# HEC293 NoST

In [6]:
roi_dir = '/home/dhruv/Cybergut/Data/09-10-18/raw/roi/'
mask_dir = '/home/dhruv/Cybergut/Data/09-10-18/raw/masks/'

rois = glob.glob(os.path.join(roi_dir, '*.zip'))

In [7]:
rois

['/home/dhruv/Cybergut/Data/09-10-18/raw/roi/A - 1(fld 06 wv TL-Brightfield - dsRed).zip',
 '/home/dhruv/Cybergut/Data/09-10-18/raw/roi/B - 3(fld 08 wv TL-Brightfield - dsRed).zip',
 '/home/dhruv/Cybergut/Data/09-10-18/raw/roi/A - 1(fld 01 wv TL-Brightfield - dsRed).zip',
 '/home/dhruv/Cybergut/Data/09-10-18/raw/roi/A - 1(fld 08 wv TL-Brightfield - dsRed).zip',
 '/home/dhruv/Cybergut/Data/09-10-18/raw/roi/A - 1(fld 03 wv TL-Brightfield - dsRed).zip',
 '/home/dhruv/Cybergut/Data/09-10-18/raw/roi/B - 3(fld 07 wv TL-Brightfield - dsRed).zip',
 '/home/dhruv/Cybergut/Data/09-10-18/raw/roi/B - 3(fld 14 wv TL-Brightfield - dsRed).zip',
 '/home/dhruv/Cybergut/Data/09-10-18/raw/roi/B - 3(fld 01 wv TL-Brightfield - dsRed).zip',
 '/home/dhruv/Cybergut/Data/09-10-18/raw/roi/A - 1(fld 04 wv TL-Brightfield - dsRed).zip',
 '/home/dhruv/Cybergut/Data/09-10-18/raw/roi/A - 1(fld 07 wv TL-Brightfield - dsRed).zip']

In [8]:
for r in rois:
    base = os.path.basename(r)
    base = base.replace('.zip', '.png')
    img = roi.create_color_mask(roi.get_polygons(r), 2048, 5, False)
    cv2.imwrite(os.path.join(mask_dir, base), img)